In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import dash_table
import pandas as pd
from collections import OrderedDict
import os
import json
import requests

In [2]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, './style.css'],
                    meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])

In [3]:
pop_model_params = {}
pop_model_params['LogReg'] = [{"name": "LogReg-", "description": "Using logistic regression to ", 
                               "model_selection_algorithm": "GridSearch", "executable_entrypoint": },
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'},
                              
                              {'name': 'Regularization Type', 'param_type': 'hp_choice', 'choices': 'l1,l2', 
                               'min': 'None', 'max': 'None', 'q': 'None', 'dtype': 'dtype_str'},
                             
                              {'name': 'Regularization Strength', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -3, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'}]

pop_model_params['MLP'] = [{"name": "MLP-", "description": "Using multilayer perceptron to ", 
                               "model_selection_algorithm": "GridSearch"},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'},
                              
                              {'name': 'Regularization Type', 'param_type': 'hp_choice', 'choices': 'l1,l2', 
                               'min': 'None', 'max': 'None', 'q': 'None', 'dtype': 'dtype_str'},
                             
                              {'name': 'Regularization Strength', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -3, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                          
                              {'name': 'Drop Out', 'param_type': 'uniform', 'choices': 'None', 
                               'min': 0.0, 'max': 0.5, 'q': 'None', 'dtype': 'dtype_float'}]

pop_model_params['CNN'] = [{"name": "CNN-", "description": "Using convolutional neural network to ", 
                               "model_selection_algorithm": "GridSearch"},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'},
                              
                              {'name': 'Regularization Type', 'param_type': 'hp_choice', 'choices': 'l1,l2', 
                               'min': 'None', 'max': 'None', 'q': 'None', 'dtype': 'dtype_str'},
                             
                              {'name': 'Regularization Strength', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -3, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                          
                              {'name': 'Drop Out', 'param_type': 'uniform', 'choices': 'None', 
                               'min': 0.0, 'max': 0.5, 'q': 'None', 'dtype': 'dtype_float'}]

pop_model_params['ResNet'] = [{"name": "ResNet-", "description": "Using ResNet to ", 
                               "model_selection_algorithm": "GridSearch"},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'},
                             
                              {'name': 'Drop Out', 'param_type': 'uniform', 'choices': 'None', 
                               'min': 0.0, 'max': 0.5, 'q': 'None', 'dtype': 'dtype_float'}]

pop_model_params['Bert'] = [{"name": "Bert-", "description": "Using distill BERT to ", 
                               "model_selection_algorithm": "GridSearch"},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'}]


In [4]:
sample_model = [{'name': 'model', 'lr': '1', 'C': '1', 'penalty': 'l1'},
                {'name': 'mode2', 'lr': '1', 'C': '1', 'penalty': 'l2'},
                {'name': 'mode3', 'lr': '1', 'C': '0.1', 'penalty': 'l1'},
                {'name': 'mode4', 'lr': '1', 'C': '0.1', 'penalty': 'l2'},
                {'name': 'mode5', 'lr': '0.1', 'C': '1', 'penalty': 'l1'},
                {'name': 'mode6', 'lr': '0.1', 'C': '1', 'penalty': 'l2'},
                {'name': 'mode7', 'lr': '0.1', 'C': '0.1', 'penalty': 'l1'},
                {'name': 'mode8', 'lr': '0.1', 'C': '0.1', 'penalty': 'l2'}]

In [5]:
models = [{'Act': 'None', 'Clone': 'Clone', 'name': 'model 1', 'lr': '1'},
          {'Act': 'None', 'Clone': 'Clone', 'name': 'model 2', 'lr': '0.1'},
          {'Act': 'None', 'Clone': 'Clone', 'name': 'model 3', 'lr': '0.01'},
          {'Act': 'None', 'Clone': 'Clone', 'name': 'model 4', 'lr': '0.001'}]

In [6]:
EXPERIMENT_ELEMENT = (
    "name", "description", "model_selection_algorithm", "max_num_models", "feature_columns", "label_columns", 
    "max_train_epochs", "data_store_prefix_path", "executable_entrypoint"
)
EXPERIMENT_TYPE = (
    "string", "string", "string", "string", "string", "int", "string", "string"
)
describe_list = (
    "Name of experiment (string type)", 
    "Description (optional)", 
    "Hyperparameters search strategy (string type)",
    "Name of features (string type, comma seperated)", 
    "Name of label (string type)", 
    "Maximum training epochs (integer type)",
    "Prefix training data path (string type)", 
    "Estimator function name (<module_name>:<function_name>)"
)

In [7]:
# App layout
app.layout = dbc.Container([

    dbc.Navbar(
        [
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [dbc.Col(dbc.NavbarBrand("Human In Loop Cerebro Training", className="ml-2"))],
                align="center",
                no_gutters=True,
            )
        ],
        color="dark",
        dark=True, style = {'width': '100%'}
    ),
    
#     dbc.Row( dbc.Col(html.H1("Human In Loop Cerebro Training", className='text-center'))),

#     html.Hr(),
    
    # *************************************************************************************************
    #
    #     Model selection section
    #
    # *************************************************************************************************
    
    dbc.Row([dbc.Col(html.H3("Upload a model script file OR choose a popular model", className='text-center'))]),
    
    dbc.Row( [
        dbc.Col(
            dcc.Upload(
                id='upload-script',
                children = ['Drag and Drop or ',
                    html.A('Select a File', style ={ 'text-decoration': 'underline'})
                ], 
                style={
                    'width': '100%',
                    'height': '130px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center'
                },
                multiple=True
            ),
            className='column_left'
        ), 
        dbc.Col([
            html.H6("Select a popular model:"),
            dcc.Dropdown(id="slct_pop_model",
                 options=[
                     {"label": "Logistic Regression", "value": "LogReg"},
                     {"label": "MLP", "value": "MLP"},
                     {"label": "CNN", "value": "CNN"},
                     {"label": "ResNet-50", "value": "ResNet"},
                     {"label": "DistillBERT", "value": "Bert"}],
                 multi=False,
                 value="LogReg"
            ),
            html.Button('Select this model', id='slct_model', n_clicks=0, style={"margin-top": "15px"})
        ])
    ]),
    
    html.Div(id='script_response', children=[]),
    
    html.Hr(),
    
    # *************************************************************************************************
    #
    #     Experiment setup section
    #
    # *************************************************************************************************
    
    html.H3("Setup experiment"),
    
    dbc.Row([
        dbc.Col(
            [
                dbc.Row([html.Div(Describe)]) for Describe in describe_list
            ], width=8
        ),
        dbc.Col(
            [ 
                dbc.Row([dcc.Input(
                    id="exp_{}".format(e),
                    type=t,
                    placeholder="{}:({})".format(e,t),
                    style={
                    'width': '200px',
                    'height': '25px'}
                )])
                for e,t in zip(EXPERIMENT_ELEMENT,EXPERIMENT_TYPE)
            ], width=4
        )
    ]),
        
    html.H5("Hyperparameter grid table:"),
      
    dash_table.DataTable(
        id='datatable-params',
        data=pop_model_params['LogReg'],
        columns=[
            {"name": 'Name', 'id': 'name', "selectable": True},
            {"name": "Hyperparameter Type", "id": "param_type", 'presentation': 'dropdown'},
            {"name": "Choices", "id": "choices", "selectable": True},
            {"name": "Min Value", "id": "min", "selectable": True},
            {"name": "Max Value", "id": "max", "selectable": True},
            {"name": "Sample Number", "id": "q", "selectable": True},
            {"name": "Data Type", "id": "dtype", 'presentation': 'dropdown'}
        ],
        editable=True,
        filter_action="native",     # allow filtering of data by user ('native') or not ('none')
        page_current=0,             # page number that user is on
        page_size=20,                # number of rows visible per page
        style_cell={                # ensure adequate header width when text is shorter than cell's text
            'width': 'auto', 'height': 'auto', #'whiteSpace': 'normal'
        },
        style_cell_conditional=[    # align text columns to left. By default they are aligned to right
            {
                'if': {'column_id': c},
                'textAlign': 'left'
            } for c in ['name', 'param_type', "choices", "min", "max", "q", "dtype"]
        ],
        style_data={                # overflow cells' content into multiple lines
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        dropdown={
            'param_type': {
                'options': [
                    {'label': 'categorical', 'value': 'hp_choice'},
                    {'label': 'uniform', 'value': 'hp_uniform'},
                    {'label': 'quantum uniform', 'value': 'hp_quniform'},
                    {'label': 'log uniform', 'value': 'hp_loguniform'},
                    {'label': 'quantum log uniform', 'value': 'hp_qloguniform'}
                ],
                'value':'hp_choice'
            },
            'dtype': {
                'options': [
                    {'label': 'string', 'value': 'dtype_str'},
                    {'label': 'integer', 'value': 'dtype_int'},
                    {'label': 'float', 'value': 'dtype_float'}
                ],
                'value':'dtype_str'
            }
        }
    ),    
    
    html.Button('Add New Parameter', id='add_param', n_clicks=0),
        
    html.H5("Please check info above and launch the experiment by clicking the button below:"),
    html.Button('Launch This Experiment', id='add_exp', n_clicks=0, 
                    style = {'height': '40px', 'textAlign': 'center',
                             'color': 'white', 'background-color': 'DeepSkyBlue', 'font-weight': 'bold'}),      
    html.Div(id='param_output', children=[]),
    html.Hr(),
 
    # *************************************************************************************************
    #
    #     Models inspection section
    #
    # *************************************************************************************************
    
    html.H3("Models"),
      
    dbc.Row([
        dbc.Col(
            [dbc.Row(html.H5("Action"), style={"margin-top": "15px", "margin-bottom": "18px"})]
            +
            [
                dbc.Row([html.Button('Resume', id='resume_{}'.format(m['name']), n_clicks=0),
                         html.Button('Delete', id='delete_{}'.format(m['name']), n_clicks=0),
                         html.Button('Stop', id='stop_{}'.format(m['name']), n_clicks=0),
                         html.Button('Clone', id='clone_{}'.format(m['name']), n_clicks=0)]) for m in sample_model
            ]
        ),
        dbc.Col(
            dash_table.DataTable(
                id='datatable-interactivity',
                data=sample_model,
                columns=[
                    {"name": "Model Name", "id": "name", "selectable": True },
                    {"name": "Lr", "id": "lr", "selectable": True},
                    {"name": 'Penalty', 'id': 'penalty', "selectable": True},
                    {"name": 'C', 'id': 'C', "selectable": True}
                ],
                editable=True,
                filter_action="native",     # allow filtering of data by user ('native') or not ('none')
                page_current=0,             # page number that user is on
                page_size=10,                # number of rows visible per page
                style_cell={                # ensure adequate header width when text is shorter than cell's text
                    'minWidth': 95, 'maxWidth': 95, 'width': 95
                },
                style_cell_conditional=[    # align text columns to left. By default they are aligned to right
                    {
                        'if': {'column_id': c},
                        'textAlign': 'left'
                    } for c in ['name', 'lr', 'penalty', 'C']
                ],
                style_data={                # overflow cells' content into multiple lines
                    'whiteSpace': 'normal',
                    'height': 'auto'
                }
            )
        )
    ]),
    html.Br(),
    
    dcc.Input(id="model_id_to_change", type="string", placeholder="Model Name",
                    style={'width': '200px', 'height': '25px'}),
    html.Br(),
    dbc.Row([html.Button('Resume', id='resume_', n_clicks=0),
                         html.Button('Delete', id='delete_', n_clicks=0),
                         html.Button('Stop', id='stop_', n_clicks=0),
                         html.Button('Clone', id='clone_', n_clicks=0)]),
    
    html.Hr(),
    html.Iframe(src="http://localhost:6006/", style={"height": "1067px", "width": "100%"}),
    html.Hr(),
    
    html.H5("Please check info above and submit these model changes by clicking the button below:"),
    html.Button('Submit', id='submit-val', n_clicks=0,
                   style = {'width': '150px', 'height': '40px', 'textAlign': 'center',
                             'color': 'black', 'background-color': 'Chartreuse', 'font-weight': 'bold'}),
    
    html.Br(),
    html.Div(id='output_container', children=[]),
    html.Br(),

])

In [8]:
@app.callback([Output('upload-script', 'contents'),
              Output('script_response', 'children')],
              Input('upload-script', 'contents'),
              State('upload-script', 'filename'))
def upload_script(list_of_contents, list_of_names):
    printout = None
    if list_of_contents is not None:
        print("contents:", list_of_contents)
        print("filename:", list_of_names)
        script_post_url = "http://localhost:8889/api/scripts/upload"
        r = requests.post(script_post_url, files={'file': list_of_contents[0]})
        print(r)
        printout = str(r)
    return None, printout

In [9]:
@app.callback(
    [Output(component_id='datatable-params', component_property='data'),
     Output(component_id='add_param', component_property='n_clicks'), 
     Output(component_id='slct_model', component_property='n_clicks'),
     Output(component_id='datatable-params', component_property='editable')],
    [Input(component_id='add_param', component_property='n_clicks'),
     Input(component_id='slct_model', component_property='n_clicks')],
    [State('datatable-params', 'data'), State('datatable-params', 'editable'), State('slct_pop_model', 'value')]
)
def add_param(add_param_n_clicks, slct_model_n_click, data, editable, value):
    print('Button has been clicked.')   
    
    if add_param_n_clicks > 0:
        data.append({'param_name': 'name?', 'param_type': 'categorical', 'param_values': 'None', 'param_min_val': 0, 
                      'param_max_val': 0, 'param_count': 0, 'param_base': 0})
    
    if slct_model_n_click > 0:
        data = pop_model_params[value]
        editable = False
        
    return data, 0, 0, editable

In [10]:
@app.callback(
    [Output(component_id='datatable-interactivity', component_property='data'),
     Output(component_id='datatable-interactivity', component_property='columns'),
     Output(component_id='add_exp', component_property='n_clicks')],
    [Input(component_id='add_exp', component_property='n_clicks')],
    [State('datatable-params', 'data'), State(component_id='datatable-interactivity', component_property='data'),
     State(component_id='datatable-interactivity', component_property='columns')]
)
def add_exp(n_clicks_add_exp, data, return_data, columns):
    print('Button has been clicked.')
    if n_clicks_add_exp > 0:
        exp_post_url = "http://localhost:8889/api/experiments/"
        r = requests.post(exp_post_url, json=payload)
        print(r)
        print(r.json())
        return_data = models

        columns=[
            {"name": 'Action', 'id': 'Act', 'presentation': 'dropdown'},
            {"name": 'Clone', 'id': 'Clone', 'presentation': 'button'},
            {"name": "Model Name", "id": "name", "selectable": True},
            {"name": "Lr", "id": "lr", "selectable": True}
        ]
        
        
    return [return_data, columns, 0]

In [11]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    Output(component_id='output_container', component_property='children'),
    [Input(component_id='submit-val', component_property='n_clicks')],
    [State('num_epoch', 'value'), State('datatable-interactivity', 'data')]
)
def update_models(n_clicks, value, data):
    print('Button has been clicked.')
    container = "Button has been clicked {} time".format(n_clicks)
    if n_clicks > 0:
        print(value)
        print(type(value))
        print(data)

    return container

In [12]:
payload = {
  "name": "sample_svm",
  "description": "sample of svm",
  "param_defs": [
    {
      "name": "lr",
      "param_type": "log",
      "values": "string",
      "min_val": -3,
      "max_val": 0,
      "count": 4,
      "base": 10
    }
  ],
  "feature_columns": "lr",
  "label_columns": "0/1",
  "max_train_epochs": 10,
  "training_data_prefix_path": "/some_path",
  "executable_entrypoint": "script:estimator_gen_fn"
}

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False, port=8051)

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Mar/2021 18:08:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:08:06] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:08:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:08:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:08:06] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:08:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:08:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:08:06] "POST /_dash-update-component HTTP/1.1" 200 -


Button has been clicked.
Button has been clicked.


127.0.0.1 - - [09/Mar/2021 18:08:20] "POST /_dash-update-component HTTP/1.1" 200 -


Button has been clicked.
{'param_name': 'Learning Rate', 'param_type': 'log_uni', 'param_values': 'None', 'param_min_val': -4, 'param_max_val': 1, 'param_count': 6, 'param_base': 10}
{'param_name': 'Batch Size', 'param_type': 'log_uni', 'param_values': 'None', 'param_min_val': 5, 'param_max_val': 8, 'param_count': 4, 'param_base': 2}
{'param_name': 'Drop Out', 'param_type': 'uniform', 'param_values': 'None', 'param_min_val': 0.0, 'param_max_val': 0.5, 'param_count': 'None', 'param_base': 'None'}


...

Button has been clicked.
{'param_name': 'Learning Rate', 'param_type': 'log_uni', 'param_values': 'None', 'param_min_val': -4, 'param_max_val': 1, 'param_count': 6, 'param_base': 10}
{'param_name': 'Batch Size', 'param_type': 'log_uni', 'param_values': 'None', 'param_min_val': 5, 'param_max_val': 8, 'param_count': 4, 'param_base': 2}
{'param_name': 'Drop Out', 'param_type': 'uniform', 'param_values': 'None', 'param_min_val': 0.0, 'param_max_val': 0.5, 'param_count': 'None', 'param_base': 'None'}


127.0.0.1 - - [09/Mar/2021 18:10:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:18] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:18] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:18] "POST /_dash-update-component HTTP/1.1" 200 -


Button has been clicked.
Button has been clicked.


127.0.0.1 - - [09/Mar/2021 18:14:22] "POST /_dash-update-component HTTP/1.1" 200 -


Button has been clicked.
{'param_name': 'Learning Rate', 'param_type': 'log_uni', 'param_values': 'None', 'param_min_val': -4, 'param_max_val': 1, 'param_count': 6, 'param_base': 10}
{'param_name': 'Batch Size', 'param_type': 'log_uni', 'param_values': 'None', 'param_min_val': 5, 'param_max_val': 8, 'param_count': 4, 'param_base': 2}
{'param_name': 'Drop Out', 'param_type': 'uniform', 'param_values': 'None', 'param_min_val': 0.0, 'param_max_val': 0.5, 'param_count': 'None', 'param_base': 'None'}


In [ ]:
[{'props': {
    'children': [
        {'props': {
             'children': {'props': {'id': 'param_name_lr', 'type': 'string', 
             'placeholder': 'lr'}, 'type': 'Input', 'namespace': 'dash_core_components'}
         }, 
         'type': 'Col', 
         'namespace': 'dash_bootstrap_components'}
    ]
  }, 
  'type': 'Row', 
  'namespace': 'dash_bootstrap_components'}, 
 
 {'props': {'children': [{'props': {'children': {'props': {'id': 'param_name_penalty', 'type': 'string', 
    'placeholder': 'penalty'}, 'type': 'Input', 'namespace': 'dash_core_components'}}, 'type': 'Col', 
    'namespace': 'dash_bootstrap_components'}]}, 'type': 'Row', 'namespace': 'dash_bootstrap_components'}, 
 {'props': {'children': [{'props': {'children': {'props': {'id': 'param_name_C', 'type': 'string', 'placeholder': 'C'},
    'type': 'Input', 'namespace': 'dash_core_components'}}, 'type': 'Col', 'namespace': 'dash_bootstrap_components'}]},
  'type': 'Row', 'namespace': 'dash_bootstrap_components'}]

## Try requests

In [15]:
script_post_url = "http://localhost:8889/api/scripts/upload"
with open('model_gen_script.py', 'rb') as f:
    r = requests.post(script_post_url, files={'file': f})

In [17]:
r

<Response [201]>

## Original Sample Page

In [12]:
import dash
import dash_html_components as html
import dash_table
import pandas as pd
from collections import OrderedDict


app = dash.Dash(__name__)

df = pd.DataFrame(OrderedDict([
    ('climate', ['Sunny', 'Snowy', 'Sunny', 'Rainy']),
    ('temperature', [13, 43, 50, 30]),
    ('city', ['NYC', 'Montreal', 'Miami', 'NYC'])
]))


app.layout = html.Div([
    dash_table.DataTable(
        id='table-dropdown',
        data=df.to_dict('records'),
        columns=[
            {'id': 'climate', 'name': 'climate', 'presentation': 'dropdown'},
            {'id': 'temperature', 'name': 'temperature'},
            {'id': 'city', 'name': 'city', 'presentation': 'dropdown'},
        ],

        editable=True,
        dropdown={
            'climate': {
                'options': [
                    {'label': i, 'value': i}
                    for i in df['climate'].unique()
                ]
            },
            'city': {
                 'options': [
                    {'label': i, 'value': i}
                    for i in df['city'].unique()
                ]
            }
        }
    ),
    html.Div(id='table-dropdown-container')
])


if __name__ == '__main__':
    app.run_server(debug=False, port=8051)

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2021 21:58:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 21:58:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 21:58:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 21:58:02] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -


## Sample Data frame

In [3]:
df = pd.DataFrame(OrderedDict([
    ('Action', ['Sunny', 'Snowy']),
    ('Model Name', ['Model 1','Model 2']),
    ('Parameter', ['0.1', '0.01'])
]))

In [4]:
df.to_dict('records')

[{'Action': 'Sunny', 'Model Name': 'Model 1', 'Parameter': '0.1'},
 {'Action': 'Snowy', 'Model Name': 'Model 2', 'Parameter': '0.01'}]

In [ ]:
@app.callback(
    Output(component_id='datatable-interactivity', component_property='data'),
    [Input(component_id='add_model', component_property='n_clicks')],
    [State('datatable-interactivity', 'data')]
)
def add_model(n_clicks, data):
    print('Button has been clicked.')
    
    if n_clicks > 0:
        data.append({'Act': 'Add', 'Name': f'Model {len(data)+1}', 'Param': 'input later'})

    return data